In [1]:
import findspark
findspark.init()
from pyspark.sql import SparkSession
import pyspark.sql.functions as f
from pyspark.sql.functions import isnan, when, count, col, lit
import pandas as pd
import numpy as np
import json
from pyspark.sql import SparkSession

In [2]:
!/mnt/miniconda/bin/pip install spark-nlp==4.2.1 --force
!/mnt/miniconda/bin/pip install sparknlp

  Using cached spark_nlp-4.2.1-py2.py3-none-any.whl (643 kB)
  Attempting uninstall: spark-nlp
    Found existing installation: spark-nlp 4.2.1
    Uninstalling spark-nlp-4.2.1:
      Successfully uninstalled spark-nlp-4.2.1


In [3]:
spark = SparkSession.builder \
        .appName("SparkNLP") \
        .config("spark.serializer", "org.apache.spark.serializer.KryoSerializer") \
        .config("spark.jars.packages", "com.johnsnowlabs.nlp:spark-nlp_2.12:4.2.1") \
    .master('yarn') \
    .getOrCreate()

Ivy Default Cache set to: /home/hadoop/.ivy2/cache
The jars for the packages stored in: /home/hadoop/.ivy2/jars
:: loading settings :: url = jar:file:/usr/lib/spark/jars/ivy-2.4.0.jar!/org/apache/ivy/core/settings/ivysettings.xml
com.johnsnowlabs.nlp#spark-nlp_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-46068cbd-0f15-417a-b29d-2179893d0607;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp_2.12;4.2.1 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	foun

In [4]:
spark

In [5]:
from sparknlp.annotator import *
from sparknlp.base import *
import sparknlp
from sparknlp.pretrained import PretrainedPipeline

In [29]:
!/mnt/miniconda/bin/pip install vaderSentiment

     |████████████████████████████████| 125 kB 29.8 MB/s eta 0:00:01


In [30]:
### Import Vader sentiment labeler 
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

In [7]:
### Reading in the data
df_sub_processed = spark.read.parquet("s3a://ppol567-llj40-bucket-3/preprocessed_submissions")
df_com_processed = spark.read.parquet("s3a://ppol567-llj40-bucket-3/preprocessed_comments")

In [8]:
### Viewing the specs
print(df_sub_processed.count())
df_sub_processed.printSchema()

print(df_com_processed.count())
df_com_processed.printSchema()

146327
root
 |-- id: string (nullable = true)
 |-- title: string (nullable = true)
 |-- created_date: string (nullable = true)
 |-- date_clean: date (nullable = true)
 |-- Live_Thread: boolean (nullable = true)
 |-- War_Dummy: boolean (nullable = true)
 |-- finished_clean: array (nullable = true)
 |    |-- element: string (containsNull = true)



16171595
root
 |-- id: string (nullable = true)
 |-- submission_id: string (nullable = true)
 |-- created_date: string (nullable = true)
 |-- date_clean: date (nullable = true)
 |-- body: string (nullable = true)
 |-- controversiality: long (nullable = true)
 |-- finished_clean: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [9]:
df_sub_processed.show(5)

+-------+--------------------+------------+----------+-----------+---------+--------------------+
|     id|               title|created_date|date_clean|Live_Thread|War_Dummy|      finished_clean|
+-------+--------------------+------------+----------+-----------+---------+--------------------+
| zuu0wk|"Ein irreparabler...|  12-25-2022|2022-12-25|      false|     true|[ein, irreparable...|
| tfxnq4|"Good luck with t...|  03-17-2022|2022-03-17|      false|    false|        [good, luck]|
| uaxrfp|"No one betrayed ...|  04-24-2022|2022-04-24|      false|    false|[betray, village,...|
| tghuvb|"Russian warship,...|  03-17-2022|2022-03-17|      false|     true|[russian, warship...|
|10imxsx|#BBTitans: Yemi s...|  01-22-2023|2023-01-22|      false|    false|[bbtitans, yemi, ...|
+-------+--------------------+------------+----------+-----------+---------+--------------------+
only showing top 5 rows



In [10]:
df_com_processed.show(5)

+-------+-------------+------------+----------+--------------------+----------------+--------------------+
|     id|submission_id|created_date|date_clean|                body|controversiality|      finished_clean|
+-------+-------------+------------+----------+--------------------+----------------+--------------------+
|hya5fg8|       t0h8af|  02-24-2022|2022-02-24|Isnt reddit alway...|               1|[isnt, reddit, an...|
|hya5fgw|       t0i53m|  02-24-2022|2022-02-24| tf are you on about|               0|                [tf]|
|hya5fkj|       t0i53m|  02-24-2022|2022-02-24|I believe it is t...|               0|[short, path, kyi...|
|hya5fl1|       t0br75|  02-24-2022|2022-02-24|Because Fox "News...|               0|[fox, news, 90, m...|
|hya5fli|       t079w7|  02-24-2022|2022-02-24|The last one make...|               0|[make, lot, sense...|
+-------+-------------+------------+----------+--------------------+----------------+--------------------+
only showing top 5 rows



In [13]:
### Creating a new column for the sentence to apply sentence embeddings and implement goal specific processing. 
df_sub_processed = df_sub_processed.withColumn("string_form", f.concat_ws(" ", col("finished_clean")))
df_com_processed = df_com_processed.withColumn("string_form", f.concat_ws(" ", col("finished_clean")))
df_com_processed.show(5)




+-------+-------------+------------+----------+--------------------+----------------+--------------------+--------------------+
|     id|submission_id|created_date|date_clean|                body|controversiality|      finished_clean|         string_form|
+-------+-------------+------------+----------+--------------------+----------------+--------------------+--------------------+
|hya5fg8|       t0h8af|  02-24-2022|2022-02-24|Isnt reddit alway...|               1|[isnt, reddit, an...|isnt reddit angry...|
|hya5fgw|       t0i53m|  02-24-2022|2022-02-24| tf are you on about|               0|                [tf]|                  tf|
|hya5fkj|       t0i53m|  02-24-2022|2022-02-24|I believe it is t...|               0|[short, path, kyi...|short path kyiv n...|
|hya5fl1|       t0br75|  02-24-2022|2022-02-24|Because Fox "News...|               0|[fox, news, 90, m...|fox news 90 milli...|
|hya5fli|       t079w7|  02-24-2022|2022-02-24|The last one make...|               0|[make, lot, sense..

In [60]:
### Only accessing live thread submissions and comments 
df_live_sub = df_sub_processed.filter(col("Live_Thread") == 1)

### Getting the live thread ids 
df_sub_live_id = df_live_sub.select("id").collect()
df_sub_live_id = [row.id for row in df_sub_id]

### Extracting the live thread comments 
df_live_com = df_com_processed.filter(col("submission_id").isin(df_sub_live_id))
df_live_com.count()

### Removing comments from subreddit moderators 
df_live_com = df_live_com.withColumn("moderator", col("string_form").rlike("(?i)moderator"))
df_live_clean = df_live_com.filter(col("moderator") == False)

In [76]:
### Model 1 twitter sentiment analyser 
MODEL_1 = 'sentimentdl_use_twitter'

documentAssembler = DocumentAssembler() \
    .setInputCol("string_form") \
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
 .setInputCols(["document"]) \
 .setOutputCol("sentence_embeddings")

sentimentdl = SentimentDLModel.pretrained(name=MODEL_1, lang="en") \
    .setInputCols(["sentence_embeddings"]) \
    .setOutputCol("sentiment")
                
sentiment_pipeline_1 = Pipeline(stages = [documentAssembler,
                                       use,
                                       sentimentdl
                                      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_twitter download started this may take some time.
Approximate size to download 11.4 MB
[OK!]


In [82]:
### Model 2 vivekn sentiment analyser
documentAssembler = DocumentAssembler() \
.setInputCol("string_form") \
.setOutputCol("document")

tokenizer = Tokenizer() \
.setInputCols(["document"]) \
.setOutputCol("tokenized")

normalizer = Normalizer() \
.setInputCols(["tokenized"]) \
.setOutputCol("normalized")

vivekn =  ViveknSentimentModel.pretrained() \
.setInputCols(["document", "normalized"]) \
.setOutputCol("vn_sentiment")

finisher = Finisher() \
.setInputCols(["vn_sentiment"]) \
.setOutputCols("sentiment")

vivek_sentiment_pipeline = Pipeline(stages = [documentAssembler,
                                       tokenizer,
                                       normalizer,
                                       vivekn,
                                       finisher
                                      ])

sentiment_vivekn download started this may take some time.
Approximate size to download 873.6 KB
[OK!]


In [77]:
### Model 3 
MODEL_3 = "sentimentdl_use_imdb"

documentAssembler = DocumentAssembler() \
    .setInputCol("string_form") \
    .setOutputCol("document")
    
use = UniversalSentenceEncoder.pretrained(name="tfhub_use", lang="en") \
 .setInputCols(["document"]) \
 .setOutputCol("sentence_embeddings")

sentimentDl = SentimentDLModel.pretrained(name = MODEL_3, lang = 'en') \
                                          .setInputCols("sentence_embeddings") \
                                          .setOutputCol("sentiment")

sentiment_pipeline_3 = Pipeline(stages = [documentAssembler,
                                       use,
                                       sentimentdl
                                      ])

tfhub_use download started this may take some time.
Approximate size to download 923.7 MB
[OK!]
sentimentdl_use_imdb download started this may take some time.
Approximate size to download 12 MB
[OK!]


In [104]:
### Training the three models on the live thread comments dataset 
## Model 1
sentiment_model_1 = sentiment_pipeline_1.fit(df_live_clean)
result_1 = sentiment_model_1.transform(df_live_clean)

## Model 2
sentiment_model_2 = vivek_sentiment_pipeline.fit(df_live_clean)
result_2 = sentiment_model_2.transform(df_live_clean)

## Model 3 
sentiment_model_3 = sentiment_pipeline_3.fit(df_live_clean)
result_3 = sentiment_model_3.transform(df_live_clean)

In [126]:
### Extracting the sentiments 
result_1 = result_1.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")
result_2 = result_2.select("id", f.explode(col("sentiment"))).withColumnRenamed("col", "sentiment")
result_3 = result_3.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")

total_count = result_2.count()


### Aggregating the results 
result_1_agg = result_1.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_2_agg = result_2.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_3_agg = result_3.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()


In [129]:
### Formatting the results 
r_1 = result_1_agg
r_1["Proportion of Comments"] = r_1["Proportion of Comments"].round(2).map("{:,.2f}%".format)
r_1

r_2 = result_2_agg
r_2["Proportion of Comments"] = r_2["Proportion of Comments"].map("{:,.2f}%".format)
r_2

r_3 = result_3_agg
r_3["Proportion of Comments"] = r_3["Proportion of Comments"].map("{:,.2f}%".format)
r_3

TypeError: can't multiply sequence by non-int of type 'float'

In [148]:
### Concatinating the results and displaying as table 
r_list = [r_1, r_2, r_3]
results_final = pd.concat(r_list)
Models = ["Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer",\
          "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", \
          "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer"]
r_f = results_final
r_f["sentiment"] = r_f["sentiment"].str.replace("na", "neutral")
r_f = r_f.assign(Model = Models)
r_f = r_f.pivot(values = "Proportion of Comments", columns = "sentiment", index = "Model")
r_f

sentiment,negative,neutral,positive
Model,,,
IMDB Sentiment Analyzer,52.54%,4.32%,43.30%
Twitter Sentiment Analyzer,52.54%,4.32%,43.30%
Vivek Sentiment Analyzer,44.78%,18.91%,36.30%


In [150]:
### Saving the table 
r_f.to_csv("sentiment_results_1.csv")

In [80]:
### Russia-Ukraine War comments excluding the live threads 
war_ids = df_sub_processed.filter("War_Dummy == true").filter("Live_Thread == false").select("id").collect()
war_ids = [row.id for row in war_ids]

### War submissions
war_submission = df_sub_processed.filter("War_Dummy == true").filter("Live_Thread == false")
print("The number of submissions pertaining to the war : ", war_submission.count())

### War Comments 
war_comment = df_com_processed.filter(col("submission_id").isin(war_ids))
print("The number of comments pertaining to the war : ", war_comment.count())

### Removing comments from subreddit moderators 
war_comment = war_comment.withColumn("moderator", col("string_form").rlike("(?i)moderator"))
war_comment_clean = war_comment.filter(col("moderator") == False)

The number of submissions pertaining to the war :  61960


23/04/04 02:33:06 WARN DAGScheduler: Broadcasting large task binary with size 1717.6 KiB


The number of comments pertaining to the war :  6359956


In [151]:
### Training on the three models on war submissions  
## Model 1
sentiment_model_1 = sentiment_pipeline_1.fit(war_submission)
result_1 = sentiment_model_1.transform(war_submission).cache()

## Model 2
sentiment_model_2 = vivek_sentiment_pipeline.fit(war_submission)
result_2 = sentiment_model_2.transform(war_submission).cache()

## Model 3 
sentiment_model_3 = sentiment_pipeline_3.fit(war_submission)
result_3 = sentiment_model_3.transform(war_submission).cache()

In [152]:
### Extracting the sentiments 
result_1 = result_1.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")
result_2 = result_2.select("id", f.explode(col("sentiment"))).withColumnRenamed("col", "sentiment")
result_3 = result_3.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")

total_count = result_2.count()


### Aggregating the results 
result_1_agg = result_1.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_2_agg = result_2.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_3_agg = result_3.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()


In [155]:
### Formatting the results 
r_1 = result_1_agg
r_1["Proportion of Comments"] = r_1["Proportion of Comments"].round(2).map("{:,.2f}%".format)
r_1

r_2 = result_2_agg
r_2["Proportion of Comments"] = r_2["Proportion of Comments"].map("{:,.2f}%".format)
r_2

r_3 = result_3_agg
r_3["Proportion of Comments"] = r_3["Proportion of Comments"].map("{:,.2f}%".format)
r_3

,sentiment,Proportion of Comments
0,positive,20.85%
1,neutral,1.25%
2,negative,77.90%


In [157]:
### Concatinating the results and displaying as table 
r_list = [r_1, r_2, r_3]
results_final = pd.concat(r_list)
Models = ["Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer",\
          "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", \
          "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer"]
r_f_1 = results_final
r_f_1["sentiment"] = r_f_1["sentiment"].str.replace("na", "neutral")
r_f_1 = r_f_1.assign(Model = Models)
r_f_1["Type"] = "Submissions"
#r_f_1 = r_f_1.pivot(values = "Proportion of Comments", columns = "sentiment", index = "Model")
r_f_1

,sentiment,Proportion of Comments,Model,Type
0,positive,20.85%,Twitter Sentiment Analyzer,Submissions
1,neutral,1.25%,Twitter Sentiment Analyzer,Submissions
2,negative,77.90%,Twitter Sentiment Analyzer,Submissions
0,positive,24.54%,Vivek Sentiment Analyzer,Submissions
1,neutral,30.27%,Vivek Sentiment Analyzer,Submissions
2,negative,45.19%,Vivek Sentiment Analyzer,Submissions
0,positive,20.85%,IMDB Sentiment Analyzer,Submissions
1,neutral,1.25%,IMDB Sentiment Analyzer,Submissions
2,negative,77.90%,IMDB Sentiment Analyzer,Submissions


In [158]:
### Training on the three models on war comments 
## Model 1
sentiment_model_1 = sentiment_pipeline_1.fit(war_comment_clean)
result_1 = sentiment_model_1.transform(war_comment_clean)

## Model 2
sentiment_model_2 = vivek_sentiment_pipeline.fit(war_comment_clean)
result_2 = sentiment_model_2.transform(war_comment_clean)

## Model 3 
sentiment_model_3 = sentiment_pipeline_3.fit(war_comment_clean)
result_3 = sentiment_model_3.transform(war_comment_clean)

In [159]:
### Extracting the sentiments 
result_1 = result_1.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")
result_2 = result_2.select("id", f.explode(col("sentiment"))).withColumnRenamed("col", "sentiment")
result_3 = result_3.select("id", f.explode(col("sentiment.result"))).withColumnRenamed("col", "sentiment")

total_count = result_2.count()


### Aggregating the results 
result_1_agg = result_1.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_2_agg = result_2.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()

result_3_agg = result_3.groupby("sentiment") \
                       .agg((f.count(col("sentiment"))/total_count*100).alias("Proportion of Comments")).toPandas()


23/04/04 05:14:25 WARN DAGScheduler: Broadcasting large task binary with size 2.8 MiB
23/04/04 05:19:54 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
23/04/04 05:44:26 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB
23/04/04 05:44:31 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
23/04/04 05:49:57 WARN DAGScheduler: Broadcasting large task binary with size 2.5 MiB
23/04/04 05:50:02 WARN DAGScheduler: Broadcasting large task binary with size 2.9 MiB
23/04/04 06:13:56 WARN DAGScheduler: Broadcasting large task binary with size 2.6 MiB


In [160]:
### Formatting the results 
r_1 = result_1_agg
r_1["Proportion of Comments"] = r_1["Proportion of Comments"].round(2).map("{:,.2f}%".format)
r_1

r_2 = result_2_agg
r_2["Proportion of Comments"] = r_2["Proportion of Comments"].map("{:,.2f}%".format)
r_2

r_3 = result_3_agg
r_3["Proportion of Comments"] = r_3["Proportion of Comments"].map("{:,.2f}%".format)
r_3

,sentiment,Proportion of Comments
0,positive,43.16%
1,neutral,4.26%
2,negative,52.68%


In [161]:
### Concatenating the results and displaying as table 
r_list = [r_1, r_2, r_3]
results_final = pd.concat(r_list)
Models = ["Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer", "Twitter Sentiment Analyzer",\
          "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", "Vivek Sentiment Analyzer", \
          "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer", "IMDB Sentiment Analyzer"]
r_f_2 = results_final
r_f_2["sentiment"] = r_f_2["sentiment"].str.replace("na", "neutral")
r_f_2 = r_f_2.assign(Model = Models)
r_f_2["Type"] = "Comments"
r_f_2

,sentiment,Proportion of Comments,Model,Type
0,positive,43.16%,Twitter Sentiment Analyzer,Comments
1,neutral,4.26%,Twitter Sentiment Analyzer,Comments
2,negative,52.68%,Twitter Sentiment Analyzer,Comments
0,positive,36.18%,Vivek Sentiment Analyzer,Comments
1,neutral,17.10%,Vivek Sentiment Analyzer,Comments
2,negative,46.72%,Vivek Sentiment Analyzer,Comments
0,positive,43.16%,IMDB Sentiment Analyzer,Comments
1,neutral,4.26%,IMDB Sentiment Analyzer,Comments
2,negative,52.68%,IMDB Sentiment Analyzer,Comments


In [164]:
rf_list = [r_f_1, r_f_2]
rf_final = pd.concat(rf_list)
table_2 = rf_final 
table_2 = table_2.pivot(values = "Proportion of Comments", columns = "sentiment", index = ["Model", "Type"])
table_2

sentiment                              negative neutral positive
Model                      Type                                 
IMDB Sentiment Analyzer    Comments      52.68%   4.26%   43.16%
                           Submissions   77.90%   1.25%   20.85%
Twitter Sentiment Analyzer Comments      52.68%   4.26%   43.16%
                           Submissions   77.90%   1.25%   20.85%
Vivek Sentiment Analyzer   Comments      46.72%  17.10%   36.18%
                           Submissions   45.19%  30.27%   24.54%

In [165]:
table_2.to_csv("sentiment_results_2.csv")

In [166]:
spark.stop()